#Final Report

# Title

# Abstract

# Introduction

The dataset we use is "Skin Cancer MNIST: Ham10000". Skin Cancer MNIST: HAM10000 is a dataset collected dermatoscopic images acquired from different populations. The dataset contains 10015 dermatoscopic image including a representative collection of all important diagnotic categories in the realm of pigmented lesions. We want to know if we can build an application that can distinguish the different skin condition utilizing machine learning.

# Background

# Data

## The Dataset

<br>HAM10000 dataset ("Human Against Machine with 10000 training images") is a collection of dermatoscopic images of pigmented lesions from various source.

<br>The dataset contains 10015 dermatoscopic images including a representative collection of all important diagnotic categories in the realm of pigment lesions.

<br>There are 7 different types of skin conditions:
- Melanocytic nevi (nv)
- Melanoma (mel)
- Benign keratosis-like lesions (bkl)
- Basal cell carcinoma (bcc)
- Actinic keratoses (akiec)
- Vascular lesions (vasc)
- Dermatofibroma (df)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

The main csv file, HAM10000_metadata.csv, includes the following information for all the image data.
- Lesion_id
- Image_id
- Dx
- Dx_type
- Age
- Sex
- Localization

In [2]:
df = pd.read_csv('../content/drive/MyDrive/DSE_I2100/data/HAM10000_metadata.csv')

In [3]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


We can notice that only age has missing data.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
dtypes: float64(1), object(6)
memory usage: 547.8+ KB


In [6]:
df.describe()

,age
count,9958.000000
mean,51.863828
std,16.968614
min,0.000000
25%,40.000000
50%,50.000000
75%,65.000000
max,85.000000


## Data Cleaning

We use isnull() to get a list of all the missing data.

In [7]:
df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age             57
sex              0
localization     0
dtype: int64

We will use fillna() to fill all the missing age with mean age.

In [8]:
df['age'].fillna(int(df['age'].mean()),inplace=True)

We confirm that there is not more missing data.

In [9]:
df.isnull().sum()

lesion_id       0
image_id        0
dx              0
dx_type         0
age             0
sex             0
localization    0
dtype: int64

## EDA (Exploratory Data Analysis)

We will perform EDA to visualize data, detect errors and outliers, discover relationships and patterns.

# Methods

# Evaluation

# Conclusion

# Attribution

# Bibliography

# Appendix